In [29]:
import tensorflow as tf
import numpy as np

In [30]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [31]:
import numpy as np
from tensorflow.keras import layers,models

Train Directory 

In [32]:
train_dir = '/Users/uvaishnav/osteoscarcoma_evaluation_project/artifacts/train'

Defining fixed Variables

In [33]:
img_height = 224
img_width = 224
batch_size = 32
num_classes =3

Load Images

In [34]:
datagen = ImageDataGenerator(
    rescale = 1/255.0
)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size = (img_height,img_width),
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle = True
)

Found 800 images belonging to 3 classes.


Adaptive Normalization

Instead of using BatchNormalization, which normalizes each feature map based on the statistics computed over the entire batch, it is prefered to to use adaptive normalization technique similar to SPADE. 
his allows adaptive Normalization based on class labels.(meaning it can adapt normalization parameters based on the semantic content of the input)

In [35]:
class SPADE(layers.Layer):
    def __init__(self):                  ## This defines a custom Keras layer named SPADE. It inherits from the layers.Layer class, which is the base class for all Keras layers. 
        super(SPADE,self).__init__()
    
    def build(self,input_shape):
        input_shape = input_shape[0].as_list()
        print(input_shape)
        self.gama_fc = layers.Dense(input_shape[-1],use_bias=False)     # Scale Parameter
        self.beta_fc = layers.Dense(input_shape[-1],use_bias=False)     # Shift Parameter

    def call(self,inputs):
        x, cls_info = inputs
        gama = self.gama_fc(cls_info)
        beta = self.beta_fc(cls_info)

        return gama[:,None,None] * x + beta[:,None,None]

Define Generator

In [36]:
def build_generator(latent_dim=100,num_classes=num_classes):
    generator_input = layers.Input(shape=(latent_dim,))
    labels_input = layers.Input(shape=(num_classes,))
    combined_input = layers.concatenate([generator_input,labels_input])

    x = layers.Dense(7*7*512, use_bias=False)(combined_input)
    x = layers.Reshape((7,7,512))(x)
    x = SPADE()([x,labels_input])
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv2DTranspose(512, 5, strides=2, padding='same', use_bias=False)(x)
    x = SPADE()([x,labels_input])
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv2DTranspose(256, 3, strides=2, padding='same', use_bias=False)(x)
    x = SPADE()([x,labels_input])
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv2DTranspose(256, 3, strides=2, padding='same', use_bias=False)(x)
    x = SPADE()([x,labels_input])
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv2DTranspose(64, 3, strides=2, padding='same', use_bias=False)(x)
    x = SPADE()([x,labels_input])
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv2DTranspose(32, 4, strides=2, padding='same', use_bias=False)(x)
    x = SPADE()([x,labels_input])
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv2DTranspose(3,7,padding='same',activation='tanh',use_bias=False)(x)



    generator = models.Model([generator_input, labels_input],x)
    return generator




Gettig generator Summary (test)

In [37]:
test_gen = build_generator()
test_gen.summary()

[None, 7, 7, 512]
[None, 14, 14, 512]
[None, 28, 28, 256]
[None, 56, 56, 256]
[None, 112, 112, 64]
[None, 224, 224, 32]
Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_22 (InputLayer)       [(None, 100)]                0         []                            
                                                                                                  
 input_23 (InputLayer)       [(None, 3)]                  0         []                            
                                                                                                  
 concatenate_10 (Concatenat  (None, 103)                  0         ['input_22[0][0]',            
 e)                                                                  'input_23[0][0]']            
                                                                       

Define Descriminator

In [40]:
def build_discriminator(input_shape=(224,224,3),num_classes=num_classes):
    discriminator_input = layers.Input(shape=input_shape)
    labels_input = layers.Input(shape=(num_classes,))

    label_resized = layers.Reshape((1, 1, num_classes))(labels_input)                  # making label size consistent to image size
    label_resized = tf.image.resize(label_resized, (input_shape[0], input_shape[1]))

    combined_input = layers.Concatenate(axis=-1)([discriminator_input, label_resized])

    x = layers.Conv2D(64,3,strides=2,padding='same')(combined_input)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(0.5)(x)

    x = layers.Conv2D(128,3,strides=2,padding='same')(x)
    x = SPADE()([x,labels_input])
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(0.5)(x)

    x = layers.Conv2D(256,3,strides=2,padding='same')(x)
    x = SPADE()([x,labels_input])
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(0.5)(x)

    x = layers.Conv2D(512,3,strides=2,padding='same')(x)
    x = SPADE()([x,labels_input])
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(0.5)(x)

    x = layers.Conv2D(512,3,strides=2,padding='same')(x)
    x = SPADE()([x,labels_input])
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(0.5)(x)

    x = layers.Flatten()(x)

    x = layers.Dense(120,activation='relu')(x)
    x = layers.Dense(84,activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(1,activation='sigmoid')(x)

    discriminator = models.Model([discriminator_input, labels_input], x)

    return discriminator
    

Getting discriminator Summary (Test)

In [41]:
disc_test = build_discriminator()
disc_test.summary()

[None, 56, 56, 128]
[None, 28, 28, 256]
[None, 14, 14, 512]
[None, 7, 7, 512]
Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_27 (InputLayer)       [(None, 3)]                  0         []                            
                                                                                                  
 reshape_12 (Reshape)        (None, 1, 1, 3)              0         ['input_27[0][0]']            
                                                                                                  
 input_26 (InputLayer)       [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 tf.image.resize_10 (TFOpLa  (None, 224, 224, 3)          0         ['reshape_12[0][0]']          
 mbda)        